In [1]:
import pandas as pd
import json
import os
import numpy as np
import requests
import sqlalchemy as sql
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from pathlib import Path
import datetime
%matplotlib inline

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set response URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
ltc_url = "https://api.alternative.me/v2/ticker/Litecoin/?convert=USD"

In [4]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response,indent=4,sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 18635181,
            "id": 1,
            "last_updated": 1613956116,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 1056342635371,
                    "percent_change_1h": -1.34640710792664,
                    "percent_change_24h": 0.309960629289908,
                    "percent_change_7d": 16.3124533081465,
                    "percentage_change_1h": -1.34640710792664,
                    "percentage_change_24h": 0.309960629289908,
                    "percentage_change_7d": 16.3124533081465,
                    "price": 56537.0,
                    "volume_24h": 60347995474
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18635181,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrenci

In [5]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response,indent=4,sort_keys=True))

{
    "data": {
        "1027": {
            "circulating_supply": 114769083,
            "id": 1027,
            "last_updated": 1613956107,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 218780660227,
                    "percent_change_1h": -1.16178955489627,
                    "percent_change_24h": -1.88116793025551,
                    "percent_change_7d": 5.41000783100007,
                    "percentage_change_1h": -1.16178955489627,
                    "percentage_change_24h": -1.88116793025551,
                    "percentage_change_7d": 5.41000783100007,
                    "price": 1902.63,
                    "volume_24h": 32364718411
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 114769083,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurren

In [6]:
# Using the Python requests library, make an API call to access the current price LTC
ltc_response = requests.get(ltc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(ltc_response,indent=4,sort_keys=True))

{
    "data": {
        "2": {
            "circulating_supply": 66528353,
            "id": 2,
            "last_updated": 1613956114,
            "max_supply": 84000000,
            "name": "Litecoin",
            "quotes": {
                "USD": {
                    "market_cap": 14822875005,
                    "percent_change_1h": -1.01666413328357,
                    "percent_change_24h": -2.81878935541824,
                    "percent_change_7d": 3.91583493822493,
                    "percentage_change_1h": -1.01666413328357,
                    "percentage_change_24h": -2.81878935541824,
                    "percentage_change_7d": 3.91583493822493,
                    "price": 223.44,
                    "volume_24h": 8563756219
                }
            },
            "rank": 8,
            "symbol": "LTC",
            "total_supply": 66528353,
            "website_slug": "litecoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies

In [7]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

In [8]:
# Navigate the ETH response object to access the current price of ETH
eth_price = eth_response['data']['1027']['quotes']['USD']['price']

In [9]:
# Navigate the LTC response object to access the current price of LTC
ltc_price = ltc_response['data']['2']['quotes']['USD']['price']

In [10]:
# remember to update date
today_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=1*365)).split()[0]
three_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=3*365)).split()[0]
str(datetime.datetime.now()).split(" ")[0]
# Format current date as ISO format
today = pd.Timestamp(today_date, tz="America/New_York").isoformat()
today
# Format start and end dates as ISO format for one year period
start = pd.Timestamp(one_year_ago, tz="America/New_York").isoformat()
end = pd.Timestamp(today_date, tz="America/New_York").isoformat()
# Included for example
# # Get closing prices for FB and TWTR from the last year
# df_portfolio_year = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = start,
#     end = end
# ).df

In [11]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')
display(type(alpaca_api_key)) # test to make sure api key imported successfully, str indicates import successful
display(type(alpaca_secret_key)) # test to make sure api key imported successfully, str indicates import successful

str

str

In [12]:
# # Set the tickers to be used
# large_company_stocks=['QQQ']

# medium_company_stocks=['ARKK']

# small_company_stocks=[
# 'XSTLA',
# 'CLF',
# 'MSTR',
# 'NEO',
# 'DDD',
# 'OMCL']

# bonds = ['AGG']

In [13]:
tickers = ['AGG','QQQ']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
start_date = pd.Timestamp('2021-02-19',tz='America/New_York').isoformat() # remember to update date
end_date = pd.Timestamp('2021-02-19',tz='America/New_York').isoformat() # remember to update date

In [14]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_barset(tickers,timeframe,start=start_date,end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

AGG                                     \
                               open    high       low   close   volume   
2021-02-19 00:00:00-05:00  116.0187  116.06  115.7515  115.83  4416681   

                              QQQ                                    
                             open    high     low   close    volume  
2021-02-19 00:00:00-05:00  333.86  333.99  329.97  331.04  36173201

In [15]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = prices_df['AGG']['close'].item()

In [16]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
# spy_close_price = prices_df['SPY']['close'].item()

In [17]:
# Access the closing price for QQQ from the Alpaca DataFrame
# Converting the value to a floating point number
qqq_close_price = prices_df['QQQ']['close'].item()

In [18]:
# Access the closing price for ARKK from the Alpaca DataFrame
# Converting the value to a floating point number
# arkk_close_price = prices_df['ARKK']['close'].item()

In [19]:
# Access the closing price for O from the Alpaca DataFrame
# Converting the value to a floating point number
# o_close_price = prices_df['O']['close'].item()

In [20]:
# Access the closing price for VNQ from the Alpaca DataFrame
# Converting the value to a floating point number
# vnq_close_price = prices_df['VNQ']['close'].item()
# print(vnq_close_price)

In [21]:
# Get user info
user_info = pd.read_csv(Path('data/user_info.csv'))

In [22]:
# Start to build portfolio options
user_info

,current_age,yearly_retirement_income,retirement_age,current_savings,retirement_goal,portfolio_allocation
0,23,50,50,50,2450,"[0.0, 0.7, 0.3]"


In [23]:
age = user_info['current_age']

In [24]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user doesn't want crypto
user_info.loc[:,'portfolio_allocation'] = user_info.loc[:,'portfolio_allocation'].str.replace("'","")
portfolio_allocation = str(user_info['portfolio_allocation'])
print(portfolio_allocation)

0    [0.0, 0.7, 0.3]
Name: portfolio_allocation, dtype: object


In [25]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user does want crypto
if portfolio_allocation == '0    [0.0, 0.7, 0.3]', '0    [0.2,0.6,0.2]','0    [0.4,0.4,0.2]'

percent_bonds = portfolio_allocation[6:8]
print(percent_bonds)

SyntaxError: invalid syntax (<ipython-input-25-e595ac9df948>, line 2)

In [ ]:
percent_stocks = portfolio_allocation[10:12]
print(percent_stocks)

In [ ]:
percent_crypto = portfolio_allocation[14:16]
print(percent_crypto)

In [ ]:
aggressive_stock_amt = .7*(user_info['current_savings'])
# stock_amt = .7*(stock_int)
print(stock_amt)

In [ ]:
btc_amt = .3*(user_info['current_savings'])

In [ ]:
# GOAL: finish MVP by Tuesday, full by Thursday
# Build portfolios with each allocation
# Run monte carlos and graphs
# Finish CLI
# Test for bugs
# 